# Downloading Dependences

In [ ]:
# !curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
# !python pytorch-xla-env-setup.py --apt-packages libomp5 libopenblas-dev

In [ ]:
# !apt-get install git-lfs

In [ ]:
# !git lfs install
# !git clone https://huggingface.co/dccuchile/bert-base-spanish-wwm-uncased

In [ ]:
# !git lfs install
# !git clone https://huggingface.co/bert-base-uncased

In [ ]:
# !git lfs install
# !git clone https://huggingface.co/bert-base-multilingual-uncased

In [ ]:
# !pip install transformers==3

# Load Dependences

In [ ]:
### add NLP dependences
import pickle
import os
import torch
import pandas as pd
from scipy import stats
import numpy as np
import os

from sklearn import metrics
from sklearn.model_selection import StratifiedKFold, train_test_split

from tqdm import tqdm
from collections import OrderedDict, namedtuple
import torch.nn as nn
from torch.optim import lr_scheduler
import joblib

import logging
import transformers
from transformers import AdamW, get_linear_schedule_with_warmup, get_constant_schedule
import sys
from sklearn import metrics, model_selection

import warnings
import torch_xla
import torch_xla.debug.metrics as met
import torch_xla.distributed.data_parallel as dp
import torch_xla.distributed.parallel_loader as pl
import torch_xla.utils.utils as xu
import torch_xla.core.xla_model as xm
import torch_xla.distributed.xla_multiprocessing as xmp
import torch_xla.test.test_utils as test_utils
import warnings

from torch_xla.core.xla_model import mesh_reduce

warnings.filterwarnings("ignore")

In [ ]:
# Mount Google Drive
from google.colab import drive # import drive from google colab

ROOT = "/content/drive"     # default location for the drive
print(ROOT)                 # print content of ROOT (Optional)

drive.mount(ROOT)           # we mount the google drive at /content/drive

/content/drive
Mounted at /content/drive


# Functions

In [ ]:
class BERTBaseUncased(nn.Module):
    def __init__(self, bert_path, output_bert='pooler', NumberOfClasses=2):
        super(BERTBaseUncased, self).__init__()
        self.bert_path = bert_path
        self.bert = transformers.BertModel.from_pretrained(self.bert_path)
        self.bert_drop = nn.Dropout(0.3)
        self.output_bert = output_bert
        self.NumberOfClasses = NumberOfClasses
        self.OutPutHidden = nn.Linear(768 * 2, NumberOfClasses)
        self.OutPoller = nn.Linear(768, NumberOfClasses)

    def forward(
            self,
            ids,
            mask,
            token_type_ids
    ):
        o1, o2 = self.bert(
            ids,
            attention_mask=mask,
            token_type_ids=token_type_ids)
          
        if self.output_bert=='hidden':
          apool = torch.mean(o1, 1)
          mpool, _ = torch.max(o1, 1)
          cat = torch.cat((apool, mpool), 1)
          bo = self.bert_drop(cat)

          output = self.OutPutHidden(bo) 

        else:
          bo = self.bert_drop(o2)
          output = self.OutPoller(bo)
        
        return output

In [ ]:
class BERTDatasetTraining:
    def __init__(self, comment, targets, tokenizer, max_length):
        self.comment = comment
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.targets = targets

    def __len__(self):
        return len(self.comment)

    def __getitem__(self, item):
        comment = str(self.comment[item])
        comment = " ".join(comment.split())

        inputs = self.tokenizer.encode_plus(
            comment,
            None,
            truncation=True,
            add_special_tokens=True,
            max_length=self.max_length,
        )
        ids = inputs["input_ids"]
        token_type_ids = inputs["token_type_ids"]
        mask = inputs["attention_mask"]
        
        padding_length = self.max_length - len(ids)
        
        ids = ids + ([0] * padding_length)
        mask = mask + ([0] * padding_length)
        token_type_ids = token_type_ids + ([0] * padding_length)
        
        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[item], dtype=torch.float)
        }

In [ ]:
class TrainModel():
  def __init__(self, PathSaveFiles, BertVersion, BertPath,  OutputBert, LearningRate, BatchSize, Epochs, FileName, X_train, X_valid, y_train ,y_valid, MaxLen = 110, SaveModel=False):
    self.BertVersion = BertVersion
    self.BertPath = BertPath
    self.OutputBert = OutputBert
    self.LearningRate = LearningRate
    self.BatchSize = BatchSize
    self.Epochs = Epochs
    self.FileName = FileName
    self.X_train = X_train
    self.X_valid = X_valid
    self.y_train = y_train
    self.y_valid = y_valid
    self.NumberOfLabels = y_train.nunique()
    self.average_metrics =  'macro' if self.NumberOfLabels > 2 else 'binary'
    self.PathSaveFiles = PathSaveFiles
    self.MaxLen = MaxLen
    self.SaveModel = SaveModel


  def _run(self):
      def OpenEndSave(CurrentEpoch, module):
          if module == 'open'and CurrentEpoch == 1:
            with open(self.PathSaveFiles + self.FileName + ".pkl", "rb") as f:
              self.Results = pickle.load(f)

          elif module == 'save' and CurrentEpoch == self.Epochs:
            with open(self.PathSaveFiles + self.FileName + ".pkl",'wb') as f:
              pickle.dump(self.Results, f)


      def loss_fn(outputs, targets):
        return nn.CrossEntropyLoss()(outputs, targets)
            

      def train_loop_fn(data_loader, model, optimizer, device, scheduler=None, epoch=None):
          model.train()
          for bi, d in enumerate(data_loader):
              ids = d["ids"]
              mask = d["mask"]
              token_type_ids = d["token_type_ids"]
              targets = d["targets"]

              ids = ids.to(device, dtype=torch.long)
              mask = mask.to(device, dtype=torch.long)
              token_type_ids = token_type_ids.to(device, dtype=torch.long)
              targets = targets.to(device, dtype=torch.float)
              

              optimizer.zero_grad()
              outputs = model(
                  ids=ids,
                  mask=mask,
                  token_type_ids=token_type_ids
              )

              loss = loss_fn(outputs, targets)
              if bi % 10 == 0:
                  xm.master_print(f'bi={bi}, loss={loss}')

                  ValueLoss = loss.cpu().detach().numpy().tolist()
                  ValueLoss = xm.mesh_reduce('test_loss',ValueLoss, np.mean)
                  self.Results[self.BertVersion][self.OutputBert][self.LearningRate][self.BatchSize][epoch]['loss'].append(ValueLoss)

              loss.backward()
              xm.optimizer_step(optimizer)
              if scheduler is not None:
                  scheduler.step()

      def eval_loop_fn(data_loader, model, device):
          model.eval()
          fin_targets = []
          fin_outputs = []
          for bi, d in enumerate(data_loader):
              ids = d["ids"]
              mask = d["mask"]
              token_type_ids = d["token_type_ids"]
              targets = d["targets"]

              ids = ids.to(device, dtype=torch.long)
              mask = mask.to(device, dtype=torch.long)
              token_type_ids = token_type_ids.to(device, dtype=torch.long)
              targets = targets.to(device, dtype=torch.float)

              outputs = model(
                  ids=ids,
                  mask=mask,
                  token_type_ids=token_type_ids
              )

              targets_np = targets.cpu().detach().numpy().tolist()
              outputs = torch.argmax(outputs, dim=1)
              outputs_np = outputs.detach().cpu().numpy().tolist()

              fin_targets.extend(targets_np)
              fin_outputs.extend(outputs_np)    

          return fin_outputs, fin_targets

      # tokenizer
      tokenizer = transformers.BertTokenizer.from_pretrained(self.BertPath, do_lower_case=True)

      train_dataset = BERTDatasetTraining(
          comment=self.X_train.values,
          targets=self.y_train.values,
          tokenizer=tokenizer,
          max_length=self.MaxLen
      )

      train_sampler = torch.utils.data.distributed.DistributedSampler(
            train_dataset,
            num_replicas=xm.xrt_world_size(),
            rank=xm.get_ordinal(),
            shuffle=True)

      train_data_loader = torch.utils.data.DataLoader(
          train_dataset,
          batch_size=self.BatchSize,
          sampler=train_sampler,
          drop_last=True,
          num_workers=1
      )

      valid_dataset = BERTDatasetTraining(
          comment=self.X_valid.values,
          targets=self.y_valid.values,
          tokenizer=tokenizer,
          max_length=self.MaxLen
      )

      valid_sampler = torch.utils.data.distributed.DistributedSampler(
            valid_dataset,
            num_replicas=xm.xrt_world_size(),
            rank=xm.get_ordinal(),
            shuffle=False)

      valid_data_loader = torch.utils.data.DataLoader(
          valid_dataset,
          batch_size=16,
          sampler=valid_sampler,
          drop_last=False,
          num_workers=1
      )

      device = xm.xla_device()
      model = mx.to(device)
      

      param_optimizer = list(model.named_parameters())
      no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
      optimizer_grouped_parameters = [
          {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.001},
          {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}]

      
      lr = 0.4 * self.LearningRate * xm.xrt_world_size()
      num_train_steps = int(len(train_dataset) / self.BatchSize / xm.xrt_world_size() * self.Epochs)
      xm.master_print(f'num_train_steps = {num_train_steps}, world_size={xm.xrt_world_size()}')

      optimizer = AdamW(optimizer_grouped_parameters, lr=lr)
      scheduler = get_linear_schedule_with_warmup(
          optimizer,
          num_warmup_steps=0,
          num_training_steps=num_train_steps
      )

      best_f1, f1, best_cem, cem = 0,0,0,0

      for epoch in range(1, self.Epochs+1):
        ## print epoch
          xm.master_print(f'Epoch: {epoch} of {self.Epochs}')
        ## Open file to save results
          OpenEndSave(CurrentEpoch=epoch, module='open')

          para_loader = pl.ParallelLoader(train_data_loader, [device])
          train_loop_fn(para_loader.per_device_loader(device), model, optimizer, device, scheduler=scheduler, epoch=epoch)

          para_loader = pl.ParallelLoader(valid_data_loader, [device])
          o, t = eval_loop_fn(para_loader.per_device_loader(device), model, device)
          
          if self.NumberOfLabels == 2:
            f1 = xm.mesh_reduce('validation_f1', metrics.f1_score(t, o), np.mean)
            self.Results[self.BertVersion][self.OutputBert][self.LearningRate][self.BatchSize][epoch]['f1'].append(f1)

          else:
            self.Results[self.BertVersion][self.OutputBert][self.LearningRate][self.BatchSize][epoch]['f1_macro'].append(xm.mesh_reduce('validation_f1_macro', metrics.f1_score(t, o, average=self.average_metrics), np.mean))
            self.Results[self.BertVersion][self.OutputBert][self.LearningRate][self.BatchSize][epoch]['f1_weighted'].append(xm.mesh_reduce('validation_f1_weighted', metrics.f1_score(t, o, average='weighted'), np.mean))
            # cem = xm.mesh_reduce('validation_cem', cem_metric(t, o), np.mean)
            # self.Results[self.BertVersion][self.OutputBert][self.LearningRate][self.BatchSize][epoch]['cem'].append(xm.mesh_reduce('validation_cem', cem_metric(t, o), np.mean))

          accuracy = metrics.accuracy_score(t, o)
          accuracy = xm.mesh_reduce('test_accuracy', accuracy, np.mean)
          self.Results[self.BertVersion][self.OutputBert][self.LearningRate][self.BatchSize][epoch]['accuracy'].append(accuracy)
          self.Results[self.BertVersion][self.OutputBert][self.LearningRate][self.BatchSize][epoch]['recall'].append(xm.mesh_reduce('validation_recall', metrics.recall_score(t, o, average=self.average_metrics), np.mean))
          self.Results[self.BertVersion][self.OutputBert][self.LearningRate][self.BatchSize][epoch]['precision'].append(xm.mesh_reduce('validation_precison', metrics.precision_score(t, o, average=self.average_metrics), np.mean))
              
        ## save file with save results
          OpenEndSave(CurrentEpoch=epoch, module='save')

        ## Save model
          if self.SaveModel and epoch == self.Epochs:
            xm.save(model.state_dict(), self.PathSaveFiles + self.FileName + '.bin')
        
        ## print accuracy
          xm.master_print(f'Accuracy = {accuracy}')


#Load data

In [ ]:
# Load Data

#### Data Path
PathDataSet = '../content/drive/MyDrive/Code/EXITS/Data/'
FileDataset = 'EXIST2021_translatedTraining'
#### Load tsv as a Data Frame
df_train = pd.read_csv(PathDataSet + FileDataset + '.csv', index_col=0)

#### Create two new columns converting str labels to Num label
df_train['LabelTask1'] = df_train['task1'].apply(lambda x : 1 if x == 'sexist' else 0)
CategorisList = list(df_train.task2.unique())
CategorisList.remove('non-sexist')
CategorisList.insert(0,'non-sexist')
CategoriSexism = {CategorisList[index]: index for index in range(len(list(df_train.task2.unique())))}
df_train['LabelTask2'] = df_train['task2'].apply(lambda x : CategoriSexism[x])

#### Get columns names
TestColumnNames = list(df_train.columns)
#### Vizualise Data
df_train.head()

,test_case,id,source,language,text,task1,task2,English,Spanish,LabelTask1,LabelTask2
0,EXIST2021,1,twitter,en,"She calls herself ""anti-feminazi"" how about sh...",sexist,ideological-inequality,"She calls herself ""anti-feminazi"" how about sh...","Ella se llama ""anti-feminazi"", ¿cómo se acerca...",1,1
1,EXIST2021,2,twitter,en,"Now, back to these women, the brave and the be...",non-sexist,non-sexist,"Now, back to these women, the brave and the be...","Ahora, de vuelta a estas mujeres, la valiente ...",0,0
2,EXIST2021,3,twitter,en,"@CurvyBandida @Xalynne_B Wow, your skirt is ve...",sexist,objectification,"@CurvyBandida @Xalynne_B Wow, your skirt is ve...","@Curvybandida @xalynne_b wow, tu falda es muy ...",1,2
3,EXIST2021,4,twitter,en,@AurelieGuiboud Incredible! Beautiful!But I l...,non-sexist,non-sexist,@AurelieGuiboud Incredible! Beautiful!But I l...,@Aurelieguiboud increíble!¡Hermoso! Pero me re...,0,0
4,EXIST2021,5,twitter,en,i find it extremely hard to believe that kelly...,non-sexist,non-sexist,i find it extremely hard to believe that kelly...,Me parece extremadamente difícil creer que Kel...,0,0


In [ ]:
######################################################
############## Moddify CODE ##########################
######################################################

#### Change columns names for the train
LabelColumn = "LabelTask1"      ## "LabelTask1", "LabelTask2"
DataColumn = "English"          ## "text", "English" and "Spanish"
NewColumnsNames = {DataColumn:"Data",LabelColumn:"Label"}
df_train = df_train.rename(columns=NewColumnsNames)
# df_train = df_train.sample(frac=1).reset_index(drop=True)

#### Vizualise Data
df_train

,test_case,id,source,language,text,task1,task2,Data,Spanish,Label,LabelTask2
0,EXIST2021,1,twitter,en,"She calls herself ""anti-feminazi"" how about sh...",sexist,ideological-inequality,"She calls herself ""anti-feminazi"" how about sh...","Ella se llama ""anti-feminazi"", ¿cómo se acerca...",1,1
1,EXIST2021,2,twitter,en,"Now, back to these women, the brave and the be...",non-sexist,non-sexist,"Now, back to these women, the brave and the be...","Ahora, de vuelta a estas mujeres, la valiente ...",0,0
2,EXIST2021,3,twitter,en,"@CurvyBandida @Xalynne_B Wow, your skirt is ve...",sexist,objectification,"@CurvyBandida @Xalynne_B Wow, your skirt is ve...","@Curvybandida @xalynne_b wow, tu falda es muy ...",1,2
3,EXIST2021,4,twitter,en,@AurelieGuiboud Incredible! Beautiful!But I l...,non-sexist,non-sexist,@AurelieGuiboud Incredible! Beautiful!But I l...,@Aurelieguiboud increíble!¡Hermoso! Pero me re...,0,0
4,EXIST2021,5,twitter,en,i find it extremely hard to believe that kelly...,non-sexist,non-sexist,i find it extremely hard to believe that kelly...,Me parece extremadamente difícil creer que Kel...,0,0
...,...,...,...,...,...,...,...,...,...,...,...
6972,EXIST2021,6973,twitter,es,"Estamos igual sin pareja, pero puedes besar a ...",non-sexist,non-sexist,"We are the same without a partner, but you can...","Estamos igual sin pareja, pero puedes besar a ...",0,0
6973,EXIST2021,6974,twitter,es,2020 hijo de re mil putas,non-sexist,non-sexist,2020 son of re thousand whores,2020 hijo de re mil putas,0,0
6974,EXIST2021,6975,twitter,es,SEGURAMENTE ESTA CHICA NO COBRA EL DINERO QUE ...,non-sexist,non-sexist,Surely this girl does not charge the money I w...,SEGURAMENTE ESTA CHICA NO COBRA EL DINERO QUE ...,0,0
6975,EXIST2021,6976,twitter,es,@safetyaitana mi madre dice q va fea y i agree,sexist,objectification,@safetyaitana my mother says that goes ugly an...,@safetyaitana mi madre dice q va fea y i agree,1,2


In [ ]:
######################################################
############## Moddify CODE ##########################
######################################################

## Select Data for train
LanguageTrain = 'en'        ## 'Whole', 'en', 'es'

df_train_es = df_train.loc[df_train.loc[df_train['language']== 'es' ].index[0]:df_train.loc[df_train['language']== 'es'].index[-1]]
df_train_en = df_train.loc[df_train.loc[df_train['language']== 'en' ].index[0]:df_train.loc[df_train['language']== 'en'].index[-1]]

In [ ]:
## Get a Stratified sample of 20% of data/rows for Test (whole/es/en)
df_test_es = df_train_es.groupby(['Label']).apply(lambda x: x.sample(frac=0.2, random_state=48))
df_test_en = df_train_en.groupby(['Label']).apply(lambda x: x.sample(frac=0.2, random_state=48))
df_test_whole = pd.concat([df_test_es,df_test_en])

#Selectin the data for the Standar Train and Test
if LanguageTrain == 'whole':
  df_test = df_test_whole
elif LanguageTrain == 'es':
  df_test = df_test_es
  df_train = df_train_es
elif LanguageTrain == 'en':
  df_test = df_test_en
  df_train = df_train_en
else:
  print('wrong data')

df_test.head()

test_case    id  ... Label LabelTask2
Label                        ...                 
0     1540  EXIST2021  1541  ...     0          0
      175   EXIST2021   176  ...     0          0
      1081  EXIST2021  1082  ...     0          0
      450   EXIST2021   451  ...     0          0
      2953  EXIST2021  2954  ...     0          0

[5 rows x 11 columns]

In [ ]:
# Removing Extra Index levels
df_test_es = df_test_es.reset_index(level=0, drop=True)
df_test_en = df_test_en.reset_index(level=0, drop=True)
df_test_whole = df_test_whole.reset_index(level=0, drop=True)

# Importantt for remove index in the next cell
df_test = df_test.reset_index(level=0, drop=True)

# Checking the Data
df_test.head()

,test_case,id,source,language,text,task1,task2,Data,Spanish,Label,LabelTask2
1540,EXIST2021,1541,twitter,en,"But yeah, Chuck yer pound and tin of beans in ...",non-sexist,non-sexist,"But yeah, Chuck yer pound and tin of beans in ...","Pero sí, Chuck yer libra y lata de frijoles y ...",0,0
175,EXIST2021,176,twitter,en,@idew2 @MsButterflyyy In a tweet about systemi...,non-sexist,non-sexist,@idew2 @MsButterflyyy In a tweet about systemi...,@ IDEW2 @MSButterflyyy en un tweet sobre el ra...,0,0
1081,EXIST2021,1082,twitter,en,@Crryptiic @nagitoosimp Isn’t it just sexual h...,non-sexist,non-sexist,@Crryptiic @nagitoosimp Isn’t it just sexual h...,@Crryptiic @nagitoosimp no es solo acoso sexua...,0,0
450,EXIST2021,451,twitter,en,@realDonaldTrump these are past Speech Topics ...,non-sexist,non-sexist,@realDonaldTrump these are past Speech Topics ...,@realdonaldtrump Estos son los temas anteriore...,0,0
2953,EXIST2021,2954,twitter,en,Like I don't really care and misandry ain't re...,non-sexist,non-sexist,Like I don't really care and misandry ain't re...,Como realmente no me importa y no sea realment...,0,0


In [ ]:
# Remove the data/rows used for test set from the train set
df_train = df_train.drop(df_test.index)
df_train.head()

,test_case,id,source,language,text,task1,task2,Data,Spanish,Label,LabelTask2
0,EXIST2021,1,twitter,en,"She calls herself ""anti-feminazi"" how about sh...",sexist,ideological-inequality,"She calls herself ""anti-feminazi"" how about sh...","Ella se llama ""anti-feminazi"", ¿cómo se acerca...",1,1
1,EXIST2021,2,twitter,en,"Now, back to these women, the brave and the be...",non-sexist,non-sexist,"Now, back to these women, the brave and the be...","Ahora, de vuelta a estas mujeres, la valiente ...",0,0
2,EXIST2021,3,twitter,en,"@CurvyBandida @Xalynne_B Wow, your skirt is ve...",sexist,objectification,"@CurvyBandida @Xalynne_B Wow, your skirt is ve...","@Curvybandida @xalynne_b wow, tu falda es muy ...",1,2
3,EXIST2021,4,twitter,en,@AurelieGuiboud Incredible! Beautiful!But I l...,non-sexist,non-sexist,@AurelieGuiboud Incredible! Beautiful!But I l...,@Aurelieguiboud increíble!¡Hermoso! Pero me re...,0,0
6,EXIST2021,7,twitter,en,@Texas17761 @MomsDemand True story: Me to 18 y...,non-sexist,non-sexist,@Texas17761 @MomsDemand True story: Me to 18 y...,@ Texas17761 @momsdemand True Story: Me a 18 y...,0,0


In [ ]:
# Reset index datframes and and Remove non-sexist rows if task 2 
#### Remove non-sexist rows if task 2 
if df_train['Label'].nunique() > 2:

  df_train = df_train[df_train['Label'] != 0]
  df_train_es = df_train_es[df_train_es['Label'] != 0]
  df_train_en = df_train_en[df_train_en['Label'] != 0]

  df_test = df_test[df_test['Label'] != 0]
  df_test_whole = df_test_whole[df_test_whole['Label'] != 0]
  df_test_en = df_test_en[df_test_en['Label'] != 0]
  df_test_es = df_test_es[df_test_es['Label'] != 0]

#### Reset index
df_train = df_train.reset_index(drop=True)
df_train_es = df_train_es.reset_index(drop=True)
df_train_en = df_train_en.reset_index(drop=True)

df_test = df_test.reset_index(drop=True)
df_test_whole = df_test_whole.reset_index(drop=True)
df_test_en = df_test_en.reset_index(drop=True)
df_test_es = df_test_es.reset_index(drop=True)
 
df_test.head()

,test_case,id,source,language,text,task1,task2,Data,Spanish,Label,LabelTask2
0,EXIST2021,1541,twitter,en,"But yeah, Chuck yer pound and tin of beans in ...",non-sexist,non-sexist,"But yeah, Chuck yer pound and tin of beans in ...","Pero sí, Chuck yer libra y lata de frijoles y ...",0,0
1,EXIST2021,176,twitter,en,@idew2 @MsButterflyyy In a tweet about systemi...,non-sexist,non-sexist,@idew2 @MsButterflyyy In a tweet about systemi...,@ IDEW2 @MSButterflyyy en un tweet sobre el ra...,0,0
2,EXIST2021,1082,twitter,en,@Crryptiic @nagitoosimp Isn’t it just sexual h...,non-sexist,non-sexist,@Crryptiic @nagitoosimp Isn’t it just sexual h...,@Crryptiic @nagitoosimp no es solo acoso sexua...,0,0
3,EXIST2021,451,twitter,en,@realDonaldTrump these are past Speech Topics ...,non-sexist,non-sexist,@realDonaldTrump these are past Speech Topics ...,@realdonaldtrump Estos son los temas anteriore...,0,0
4,EXIST2021,2954,twitter,en,Like I don't really care and misandry ain't re...,non-sexist,non-sexist,Like I don't really care and misandry ain't re...,Como realmente no me importa y no sea realment...,0,0


#Load Weights

In [ ]:
def CriateFileName(BertVersionDict, NumberOfClasses):
  
  NameFile = str()
  for BertModel in BertVersionDict.keys():
    NameFile += BertModel

  if NumberOfClasses > 2:
    NameFile += 'Task2'
  else:
    NameFile += 'Task1'

  return NameFile

In [ ]:
# BertVersion = {'EnglishBert':'../content/bert-base-uncased/', 'SpanishBert':'../content/bert-base-spanish-wwm-uncased/', 'MultilingualBert':'../content/bert-base-multilingual-uncased/'}
# OutputBert = ['hidden', 'pooler']
# LearningRate = [2e-5, 3e-5, 5e-5]
# BatchSize = [32, 64]
# Epochs = 8

In [ ]:
######################################################
############## Moddify CODE - BERT model #############
######################################################

## Train Parameters
BertVersion = {'EnglishBert':'../content/bert-base-uncased/'}
OutputBert = ['hidden', 'pooler']
LearningRate = [2e-5, 3e-5, 5e-5]
BatchSize = [32, 64]
Epochs = 8

In [ ]:
## Evalute matrics
###### Task 1
MetricsTask1 = ['accuracy', 'f1', 'recall', 'precision']
###### Task 2
MetricsTask2 = ['accuracy', 'f1_macro', 'f1_weighted', 'recall', 'precision']

## Get for 'Binary' classification' task1 or 'Multilabel classifcation' task2
Metrics = MetricsTask2 if df_train['Label'].nunique() > 2 else MetricsTask1

## Criate dictinaril results
ResultsTask = { bert:{ output:{ lr:{ bat:{ epoc:{ metric:[] for metric in Metrics + ['loss']} for epoc in range(1, Epochs+1) } for bat in BatchSize} for lr in LearningRate} for output in OutputBert } for bert in BertVersion.keys() }

In [ ]:
## Where to Save Files
Path = 'drive/MyDrive/Code/EXITS/Machine-Learning-Tweets-Classification/Bert/Results/' 
BertModels = ''
for b in list(BertVersion.keys()):
  BertModels =  BertModels  + b + '_'
Folder = BertModels + LanguageTrain
Path = Path + Folder + 'DataTrain' + '/'

## Criate file to save results if it does not exist 
if not os.path.exists(Path):
  print(f'Criate folder : {Folder}' )
  print(f'Path : {Path}')
  os.makedirs(Path)

## Creating Main Parte Bert File Name
MainParteBertFileName = CriateFileName(BertVersion, NumberOfClasses=df_train['Label'].nunique()) + LanguageTrain

## Create file to save results if it does not existe
FileResults = MainParteBertFileName + 'DataTrain' + '_Results'
if not os.path.exists(Path + FileResults + '.pkl'):
  print(f'Creating File for results : {FileResults}.pkl')
  print(f'File Path : {Path}')
  with open(Path + FileResults + ".pkl",'wb') as f:
    pickle.dump(ResultsTask, f)

Creating File for results : EnglishBertTask1enDataTrain_Results.pkl
File Path : drive/MyDrive/Code/EXITS/Machine-Learning-Tweets-Classification/Bert/Results/EnglishBert_enDataTrain/


#Train

In [ ]:
### Cross Validation
for BertV, BertPath in BertVersion.items():
  for OutputB in OutputBert:

    ### Loading Bert trained weights
    mx = BERTBaseUncased(bert_path=BertPath, output_bert=OutputB, NumberOfClasses=df_train['Label'].nunique())

    for lr in LearningRate:
      for Batch in BatchSize:

        ## StratifiedKFold
        skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=48)
        fold = 1
        for train_index, valid_index in skf.split(df_train['Data'], df_train['Label']):
          X_train, X_valid = df_train.loc[train_index, 'Data'], df_train.loc[valid_index, 'Data']
          y_train, y_valid = df_train.loc[train_index, 'Label'], df_train.loc[valid_index, 'Label']

          print(f'parameters: Bertmodel: {BertV}, Output: {OutputB}, lr: {lr}, Batch: {Batch}, Totsl Num. Epochs: {Epochs}, Fold: {fold}')
          fold += 1
          MoDeL = TrainModel(PathSaveFiles = Path,
                            BertVersion=BertV,
                            BertPath=BertPath,
                            OutputBert=OutputB,
                            LearningRate=lr,
                            BatchSize=Batch,
                            Epochs=Epochs,
                            FileName= FileResults,
                            X_train=X_train, 
                            X_valid=X_valid,
                            y_train=y_train,
                            y_valid=y_valid)
        

          def _mp_fn(rank, flags):
            torch.set_default_tensor_type('torch.FloatTensor')
            a = MoDeL._run()

          FLAGS={}
          xmp.spawn(_mp_fn, args=(FLAGS,), nprocs=8, start_method='fork')

parameters: Bertmodel: EnglishBert, Output: hidden, lr: 2e-05, Batch: 32, Totsl Num. Epochs: 8, Fold: 1
num_train_steps = 77, world_size=8
Epoch: 1 of 8
bi=0, loss=0.651019275188446
Accuracy = 0.6285714285714286
Epoch: 2 of 8
bi=0, loss=0.612134575843811
Accuracy = 0.7428571428571429
Epoch: 3 of 8
bi=0, loss=0.4129303991794586
Accuracy = 0.7392857142857143
Epoch: 4 of 8
bi=0, loss=0.43036898970603943
Accuracy = 0.775
Epoch: 5 of 8
bi=0, loss=0.18055954575538635
Accuracy = 0.7785714285714286
Epoch: 6 of 8
bi=0, loss=0.1291235238313675
Accuracy = 0.7857142857142858
Epoch: 7 of 8
bi=0, loss=0.08474749326705933
Accuracy = 0.7857142857142858
Epoch: 8 of 8
bi=0, loss=0.04319370910525322
Accuracy = 0.7714285714285715
parameters: Bertmodel: EnglishBert, Output: hidden, lr: 2e-05, Batch: 32, Totsl Num. Epochs: 8, Fold: 2
num_train_steps = 77, world_size=8
Epoch: 1 of 8
bi=0, loss=0.7625068426132202
Accuracy = 0.6821428571428572
Epoch: 2 of 8
bi=0, loss=0.6414949893951416
Accuracy = 0.7357142857

In [ ]:
def AveragResults(FileName, Path):
  with open(Path + FileName + ".pkl", "rb") as f:
              Results = pickle.load(f)

  for BT, ModelBertType,  in Results.items():
    for OP, OutPut in ModelBertType.items():
      for LR, LearningRate in OutPut.items():
        for BS, BatchSize in LearningRate.items():
          for EP, Epoch in BatchSize.items():
            for Metrics, ValuesCrossValidation in  Epoch.items():
 
              # Metrics = np.mean(ValuesCrossValidation)
              Results[BT][OP][LR][BS][EP][Metrics] = np.mean(ValuesCrossValidation)
            
  with open('Average' + FileName + '.pkl','wb') as f:
    pickle.dump(Results, f)

  with open(Path + 'Average' + FileName + '.pkl','wb') as f:
    pickle.dump(Results, f)
  
  return Results

In [ ]:
## Average and Save Results
AverageResultsTask = AveragResults(FileName=FileResults, Path=Path)

In [ ]:
### create dataframe for our results
def create_Data_Frame(all_resultas):

  

  ### Criate a pandas da Frame with all results
  df_results = pd.DataFrame.from_dict({(BertType, OutpuType, LearningRate, BactSize, Epochs): all_resultas[BertType][OutpuType][LearningRate][BactSize][Epochs]
                            for BertType in all_resultas.keys()
                            for OutpuType in all_resultas[BertType].keys()
                            for LearningRate in all_resultas[BertType][OutpuType].keys()
                            for BactSize in all_resultas[BertType][OutpuType][LearningRate].keys()
                            for Epochs in all_resultas[BertType][OutpuType][LearningRate][BactSize].keys()},
                        orient='index')
  return df_results

In [ ]:
## Create a Data Frame
DfResultsTask = create_Data_Frame(all_resultas=AverageResultsTask)

### save results to a CSV file
DfResultsTask.to_csv(Path + 'Average' + FileResults + '_CSV_' + '.csv')

### See the Avarage results in the Pandas data Frame
DfResultsTask

accuracy        f1  ...  precision      loss
EnglishBert hidden 0.00002 32 1  0.632500  0.517751  ...   0.686184  0.718207
                              2  0.727500  0.728819  ...   0.694697  0.611783
                              3  0.760357  0.738647  ...   0.754953  0.504368
                              4  0.777500  0.772462  ...   0.744135  0.400660
                              5  0.777143  0.757069  ...   0.778056  0.284866
...                                   ...       ...  ...        ...       ...
            pooler 0.00005 64 4  0.715714  0.713009  ...   0.684132  0.650990
                              5  0.751786  0.747351  ...   0.718134  0.564350
                              6  0.765714  0.766917  ...   0.724512  0.454914
                              7  0.761429  0.752455  ...   0.735550  0.365019
                              8  0.761071  0.768538  ...   0.709530  0.311527

[96 rows x 5 columns]

In [ ]:
## Creating LateX Table
LabelTaskTable = FileResults
print(DfResultsTask.to_latex(multicolumn=True, multirow=False, label=LabelTaskTable))

\begin{table}
\centering
\label{EnglishBertTask1enDataTrain_Results}
\begin{tabular}{lllllrrrrr}
\toprule
            &        &         &    &   &  accuracy &        f1 &    recall &  precision &      loss \\
\midrule
EnglishBert & hidden & 0.00002 & 32 & 1 &  0.632500 &  0.517751 &  0.459445 &   0.686184 &  0.718207 \\
            &        &         &    & 2 &  0.727500 &  0.728819 &  0.791426 &   0.694697 &  0.611783 \\
            &        &         &    & 3 &  0.760357 &  0.738647 &  0.739083 &   0.754953 &  0.504368 \\
            &        &         &    & 4 &  0.777500 &  0.772462 &  0.817220 &   0.744135 &  0.400660 \\
            &        &         &    & 5 &  0.777143 &  0.757069 &  0.755135 &   0.778056 &  0.284866 \\
            &        &         &    & 6 &  0.777500 &  0.756750 &  0.750585 &   0.776875 &  0.200690 \\
            &        &         &    & 7 &  0.771786 &  0.750785 &  0.742667 &   0.774955 &  0.136298 \\
            &        &         &    & 8 &  0.771071 &

# Inference

##Train the model with Full Train dataset

In [ ]:
## 10 Best resuts
MetricForBestResults = 'f1' if df_train['Label'].nunique() > 2 else 'accuracy'
DfResultsTask.nlargest(n=10, columns= MetricForBestResults )

accuracy        f1  ...  precision      loss
EnglishBert hidden 0.00003 32 5  0.781786  0.767607  ...   0.763943  0.209806
                              4  0.780357  0.763025  ...   0.774038  0.335329
                   0.00005 64 8  0.780357  0.773331  ...   0.747310  0.254721
                   0.00002 32 4  0.777500  0.772462  ...   0.744135  0.400660
                              6  0.777500  0.756750  ...   0.776875  0.200690
                              5  0.777143  0.757069  ...   0.778056  0.284866
                   0.00005 64 7  0.773929  0.766351  ...   0.746626  0.321695
            pooler 0.00002 32 5  0.773214  0.761348  ...   0.753319  0.310749
            hidden 0.00003 32 3  0.773214  0.756826  ...   0.765053  0.462896
                              7  0.772857  0.760645  ...   0.759311  0.059705

[10 rows x 5 columns]

In [ ]:
## Get best parameters from cross-validation DataFrame 
BestResultParameters = DfResultsTask.sort_values(MetricForBestResults, ascending=False)[:1].index
print(f'Best parameters : {BestResultParameters}')

Best parameters : MultiIndex([('EnglishBert', 'hidden', 3e-05, 32, 5)],
           )


In [ ]:
## Add best parameters to variables in the final train
BertPath = BertVersion[BestResultParameters[0][0]]
BertVersion = {BestResultParameters[0][0] : BertVersion[BestResultParameters[0][0]]}
OutputBert = [BestResultParameters[0][1]]
LearningRate = [float(BestResultParameters[0][2])]
BatchSize = [int(BestResultParameters[0][3])]
Epochs = int(BestResultParameters[0][4])

In [ ]:
## Criate dictinaril results
ResultsTaskBestParameters = { bert:{ output:{ lr:{ bat:{ epoc:{ metric:[] for metric in Metrics + ['loss']} for epoc in range(1, Epochs+1) } for bat in BatchSize} for lr in LearningRate} for output in OutputBert } for bert in BertVersion.keys() }

## Create file to save results BEST Parameters
#### Create file name
FileResultsBestModel = FileResults + 'BestModel'
#### Save the file fro results BEST Parameters
with open(Path + FileResultsBestModel + ".pkl",'wb') as f:
  pickle.dump(ResultsTaskBestParameters, f)

In [ ]:
## Train with Best parameters

## Best parameters
BertV = BestResultParameters[0][0]
BertPath = BertVersion[BestResultParameters[0][0]]
OutputB = OutputBert[0]
lr = LearningRate[0]
Batch = BatchSize[0]
Epochs = Epochs

### Loading Bert trained weights
mx = BERTBaseUncased(bert_path=BertPath, output_bert=OutputB, NumberOfClasses=df_train['Label'].nunique())

## Split train and test
X_train = df_train['Data']
y_train = df_train['Label']
_, X_test, _, y_test = train_test_split(df_train['Data'], df_train['Label'], test_size=0.33, random_state=42)

print(f'parameters: Bertmodel: {BertV}, Output: {OutputB}, lr: {lr}, Batch: {Batch}, Totsl Num. Epochs: {Epochs}')
MoDeL = TrainModel(PathSaveFiles = Path,
                  BertVersion=BertV,
                  BertPath=BertPath,
                  OutputBert=OutputB,
                  LearningRate=lr,
                  BatchSize=Batch,
                  Epochs=Epochs,
                  FileName= FileResultsBestModel,
                  X_train=X_train, 
                  X_valid=X_test,
                  y_train=y_train,
                  y_valid=y_test,
                  SaveModel=True)


def _mp_fn(rank, flags):
  torch.set_default_tensor_type('torch.FloatTensor')
  a = MoDeL._run()

FLAGS={}
xmp.spawn(_mp_fn, args=(FLAGS,), nprocs=8, start_method='fork')

parameters: Bertmodel: EnglishBert, Output: hidden, lr: 3e-05, Batch: 32, Totsl Num. Epochs: 5


Exception: ignored

In [ ]:
## Average and Save Results
AverageResultsTaskBestModel = AveragResults(FileName=FileResultsBestModel, Path=Path)

In [ ]:
## Create a Data Frame
DfResultsTaskBestModel = create_Data_Frame(all_resultas=AverageResultsTaskBestModel)

### save results to a CSV file
DfResultsTaskBestModel.to_csv(Path + 'Average' + FileResultsBestModel + '_CSV_' + '.csv')

### See the Avarage results in the Pandas data Frame
DfResultsTaskBestModel

accuracy        f1  ...  precision      loss
EnglishBert hidden 0.00003 32 1  0.608553  0.237622  ...   1.000000  0.829413
                              2  0.805921  0.779498  ...   0.805057  0.630450
                              3  0.836623  0.829182  ...   0.790378  0.542547
                              4  0.895833  0.886149  ...   0.873260  0.458032
                              5  0.905702  0.897724  ...   0.879712  0.349698

[5 rows x 5 columns]

# Test

In [ ]:
class BERTDatasetTest:
    def __init__(self, comment_text, tokenizer, max_length):
        self.comment_text = comment_text
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.comment_text)

    def __getitem__(self, item):
        comment_text = str(self.comment_text[item])
        comment_text = " ".join(comment_text.split())

        inputs = self.tokenizer.encode_plus(
            comment_text,
            None,
            truncation=True,
            add_special_tokens=True,
            max_length=self.max_length,
        )
        ids = inputs["input_ids"]
        token_type_ids = inputs["token_type_ids"]
        mask = inputs["attention_mask"]
        
        padding_length = self.max_length - len(ids)
        
        ids = ids + ([0] * padding_length)
        mask = mask + ([0] * padding_length)
        token_type_ids = token_type_ids + ([0] * padding_length)
        
        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long)
        }

In [ ]:
## Bert tozenizer
tokenizer = transformers.BertTokenizer.from_pretrained(BertPath, do_lower_case=True)

In [ ]:
## Loading the best model
device = torch.device("xla")
model = BERTBaseUncased(bert_path=BertPath, output_bert=OutputB, NumberOfClasses=df_train['Label'].nunique()).to(device)
FileBestModel = Path + FileResultsBestModel + '.bin'
model.load_state_dict(torch.load(FileBestModel))
model.eval()

BERTBaseUncased(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

### Test Whole Data

In [ ]:
## Prepresing the data
valid_dataset = BERTDatasetTest(
        comment_text=df_test_whole['Data'].values,
        tokenizer=tokenizer,
        max_length=110
)

valid_data_loader = torch.utils.data.DataLoader(
    valid_dataset,
    batch_size=Batch,
    drop_last=False,
    num_workers=4,
    shuffle=False
)

In [ ]:
## Making the Inferences
with torch.no_grad():
    fin_outputs = []
    for bi, d in tqdm(enumerate(valid_data_loader)):
        ids = d["ids"]
        mask = d["mask"]
        token_type_ids = d["token_type_ids"]

        ids = ids.to(device, dtype=torch.long)
        mask = mask.to(device, dtype=torch.long)
        token_type_ids = token_type_ids.to(device, dtype=torch.long)

        outputs = model(
            ids=ids,
            mask=mask,
            token_type_ids=token_type_ids
        )

        outputs_np = outputs.detach().cpu().numpy().tolist()
        fin_outputs.extend(outputs_np) 

44it [00:11,  3.98it/s]


In [ ]:
## List with Results
fin_outputs

## create a Dataframe from List of Results
df_results = pd.DataFrame.from_records(fin_outputs)

## change columns if task2
if df_train['Label'].nunique() > 2:
  df_results = df_results.rename({0:1, 1:2, 2:3, 3:4, 4:5}, axis='columns')

## get the model inference
df_results['Inference'] = df_results.idxmax(axis=1)

## Visualize results
df_results.head()

,0,1,Inference
0,1.944694,-2.095059,0
1,1.419009,-1.424598,0
2,1.470887,-1.131224,0
3,0.565983,-0.345339,0
4,-0.671665,0.796625,1


In [ ]:
## Get rows index
df_idex = df_test_whole.loc[:,["id", "Label"]]

## Add index to the Results dataframe
df_results = df_results.join(df_idex)

### save results to a CSV file
df_results.to_csv(Path + 'ModelInfereneces_' + FileResultsBestModel + '_WholeSetTest' +'_CSV_' + '.csv')

## ## Visualize results
df_results.head()

,0,1,Inference,id,Label
0,1.944694,-2.095059,0,4516,0
1,1.419009,-1.424598,0,3573,0
2,1.470887,-1.131224,0,4232,0
3,0.565983,-0.345339,0,3782,0
4,-0.671665,0.796625,1,6357,0


In [ ]:
## caculation of performace metric
Target = df_results[df_results.columns[-1]].tolist()
Output = df_results[df_results.columns[-3]].tolist()

average_metrics = 'macro' if df_train['Label'].nunique() > 2 else 'binary'
print(f'Accuracy : {metrics.accuracy_score(Target, Output)}')
print(f'Recall : {metrics.recall_score(Target, Output, average = average_metrics)}')
print(f'Precision : {metrics.precision_score(Target, Output, average = average_metrics)}')
print(f'f1-score : {metrics.f1_score(Target, Output, average= average_metrics)}')

Accuracy : 0.7318996415770609
Recall : 0.6933333333333334
Precision : 0.7370078740157481
f1-score : 0.7145038167938932


### Test Only English

In [ ]:
## Prepresing the data
valid_dataset = BERTDatasetTest(
        comment_text=df_test_en['Data'].values,
        tokenizer=tokenizer,
        max_length=110
)

valid_data_loader = torch.utils.data.DataLoader(
    valid_dataset,
    batch_size=Batch,
    drop_last=False,
    num_workers=4,
    shuffle=False
)

In [ ]:
## Making the Inferences
with torch.no_grad():
    fin_outputs = []
    for bi, d in tqdm(enumerate(valid_data_loader)):
        ids = d["ids"]
        mask = d["mask"]
        token_type_ids = d["token_type_ids"]

        ids = ids.to(device, dtype=torch.long)
        mask = mask.to(device, dtype=torch.long)
        token_type_ids = token_type_ids.to(device, dtype=torch.long)

        outputs = model(
            ids=ids,
            mask=mask,
            token_type_ids=token_type_ids
        )

        outputs_np = outputs.detach().cpu().numpy().tolist()
        fin_outputs.extend(outputs_np) 

22it [00:05,  3.97it/s]


In [ ]:
## List with Results
fin_outputs

## create a Dataframe from List of Results
df_results = pd.DataFrame.from_records(fin_outputs)

## change columns if task2
if df_train['Label'].nunique() > 2:
  df_results = df_results.rename({0:1, 1:2, 2:3, 3:4, 4:5}, axis='columns')

## get the model inference
df_results['Inference'] = df_results.idxmax(axis=1)

## Visualize results
df_results.head()

,0,1,Inference
0,1.835893,-1.964890,0
1,0.284768,-0.076747,0
2,-1.435505,1.516354,1
3,2.074585,-2.033102,0
4,1.449182,-1.389517,0


In [ ]:
## Get rows index
df_idex = df_test_en.loc[:,["id", "Label"]]

## Add index to the Results dataframe
df_results = df_results.join(df_idex)

### save results to a CSV file
df_results.to_csv(Path + 'ModelInfereneces_' + FileResultsBestModel + '_EnglishSetTest' +'_CSV_' + '.csv')

## ## Visualize results
df_results.head()

,0,1,Inference,id,Label
0,1.835893,-1.964890,0,1541,0
1,0.284768,-0.076747,0,176,0
2,-1.435505,1.516354,1,1082,0
3,2.074585,-2.033102,0,451,0
4,1.449182,-1.389517,0,2954,0


In [ ]:
## caculation of performace metric
Target = df_results[df_results.columns[-1]].tolist()
Output = df_results[df_results.columns[-3]].tolist()

average_metrics = 'macro' if df_train['Label'].nunique() > 2 else 'binary'
print(f'Accuracy : {metrics.accuracy_score(Target, Output)}')
print(f'Recall : {metrics.recall_score(Target, Output, average = average_metrics)}')
print(f'Precision : {metrics.precision_score(Target, Output, average = average_metrics)}')
print(f'f1-score : {metrics.f1_score(Target, Output, average= average_metrics)}')

Accuracy : 0.7685589519650655
Recall : 0.7920489296636085
Precision : 0.74
f1-score : 0.7651403249630725


### Test Only Spanish

In [ ]:
## Prepresing the data
valid_dataset = BERTDatasetTest(
        comment_text=df_test_es['Data'].values,
        tokenizer=tokenizer,
        max_length=110
)

valid_data_loader = torch.utils.data.DataLoader(
    valid_dataset,
    batch_size=Batch,
    drop_last=False,
    num_workers=4,
    shuffle=False
)

In [ ]:
## Making the Inferences
with torch.no_grad():
    fin_outputs = []
    for bi, d in tqdm(enumerate(valid_data_loader)):
        ids = d["ids"]
        mask = d["mask"]
        token_type_ids = d["token_type_ids"]

        ids = ids.to(device, dtype=torch.long)
        mask = mask.to(device, dtype=torch.long)
        token_type_ids = token_type_ids.to(device, dtype=torch.long)

        outputs = model(
            ids=ids,
            mask=mask,
            token_type_ids=token_type_ids
        )

        outputs_np = outputs.detach().cpu().numpy().tolist()
        fin_outputs.extend(outputs_np) 

23it [00:04,  5.20it/s]


In [ ]:
## List with Results
fin_outputs

## create a Dataframe from List of Results
df_results = pd.DataFrame.from_records(fin_outputs)

## change columns if task2
if df_train['Label'].nunique() > 2:
  df_results = df_results.rename({0:1, 1:2, 2:3, 3:4, 4:5}, axis='columns')

## get the model inference
df_results['Inference'] = df_results.idxmax(axis=1)

## Visualize results
df_results.head()

,0,1,Inference
0,1.944694,-2.095059,0
1,1.419009,-1.424598,0
2,1.470887,-1.131224,0
3,0.565983,-0.345339,0
4,-0.671665,0.796625,1


In [ ]:
## Get rows index
df_idex = df_test_es.loc[:,["id", "Label"]]

## Add index to the Results dataframe
df_results = df_results.join(df_idex)

### save results to a CSV file
df_results.to_csv(Path + 'ModelInfereneces_' + FileResultsBestModel + '_SpanishSetTest' + '_CSV_' + '.csv')

## ## Visualize results
df_results.head()

,0,1,Inference,id,Label
0,1.944694,-2.095059,0,4516,0
1,1.419009,-1.424598,0,3573,0
2,1.470887,-1.131224,0,4232,0
3,0.565983,-0.345339,0,3782,0
4,-0.671665,0.796625,1,6357,0


In [ ]:
## caculation of performace metric
Target = df_results[df_results.columns[-1]].tolist()
Output = df_results[df_results.columns[-3]].tolist()

average_metrics = 'macro' if df_train['Label'].nunique() > 2 else 'binary'
print(f'Accuracy : {metrics.accuracy_score(Target, Output)}')
print(f'Recall : {metrics.recall_score(Target, Output, average = average_metrics)}')
print(f'Precision : {metrics.precision_score(Target, Output, average = average_metrics)}')
print(f'f1-score : {metrics.f1_score(Target, Output, average= average_metrics)}')

Accuracy : 0.6963276836158192
Recall : 0.6005747126436781
Precision : 0.7333333333333333
f1-score : 0.660347551342812


## Inference

###Load data

In [ ]:
# Load data for inference 

#### Data Path
PathDataSet = '../content/drive/MyDrive/Code/EXITS/Data/'
FileDataset = 'EXIST2021_translatedTest'
#### Load tsv as a Data Frame
df_RealData = pd.read_csv(PathDataSet + FileDataset + '.csv', index_col=0)

#### Change columns names for the train
df_RealData = df_RealData.rename(columns=NewColumnsNames)

#### Vizualise Data
df_RealData.head()

,test_case,id,source,language,text,Data,Spanish
0,EXIST2021,6978,gab,en,Pennsylvania State Rep horrifies with opening ...,Pennsylvania State Rep horrifies with opening ...,La representante del estado de Pensilvania se ...
1,EXIST2021,6979,twitter,en,"@iilovegrapes He sounds like as ass, and very ...","@iilovegrapes He sounds like as ass, and very ...","@iilovegrapes Suena como un idiota, y muy cond..."
2,EXIST2021,6980,twitter,en,"@averyangryskel1 @4ARealistParty LOL! ""This be...","@averyangryskel1 @4ARealistParty LOL! ""This be...","@ averyangryskel1 @ 4ARealistParty ¡LOL! ""¡Est..."
3,EXIST2021,6981,twitter,en,@WanderOrange @stalliontwink Rights?I mean yea...,@WanderOrange @stalliontwink Rights?I mean yea...,@WanderOrange @stalliontwink ¿Derechos? Quiero...
4,EXIST2021,6982,twitter,en,the jack manifold appreciation i’m seeing is o...,the jack manifold appreciation i’m seeing is o...,la apreciación de jack manifold que estoy vien...


In [ ]:
## Prepresing the data
valid_dataset = BERTDatasetTest(
        comment_text=df_RealData['Data'].values,
        tokenizer=tokenizer,
        max_length=110
)

valid_data_loader = torch.utils.data.DataLoader(
    valid_dataset,
    batch_size=Batch,
    drop_last=False,
    num_workers=4,
    shuffle=False
)

In [ ]:
## Making the Inferences
with torch.no_grad():
    fin_outputs = []
    for bi, d in tqdm(enumerate(valid_data_loader)):
        ids = d["ids"]
        mask = d["mask"]
        token_type_ids = d["token_type_ids"]

        ids = ids.to(device, dtype=torch.long)
        mask = mask.to(device, dtype=torch.long)
        token_type_ids = token_type_ids.to(device, dtype=torch.long)

        outputs = model(
            ids=ids,
            mask=mask,
            token_type_ids=token_type_ids
        )

        outputs_np = outputs.detach().cpu().numpy().tolist()
        fin_outputs.extend(outputs_np) 

137it [00:14,  9.69it/s]


In [ ]:
## List with Results
fin_outputs

## create a Dataframe from List of Results
df_results = pd.DataFrame.from_records(fin_outputs)

## change columns if task2
if df_train['Label'].nunique() > 2:
  df_results = df_results.rename({0:1, 1:2, 2:3, 3:4, 4:5}, axis='columns')

## get the model inference
df_results['Inference'] = df_results.idxmax(axis=1)

## Visualize results
df_results.head()

,0,1,Inference
0,1.295872,-1.186037,0
1,0.835166,-0.908593,0
2,-1.607499,1.858016,1
3,-0.430777,1.007864,1
4,2.083438,-2.189569,0


In [ ]:
## Get rows index
df_idex = df_RealData.loc[:,["id"]]

## Add index to the Results dataframe
df_results = df_results.join(df_idex)

### save results to a CSV file
df_results.to_csv(Path + 'ModelInfereneces' + FileResultsBestModel + '_RealData' + '_CSV_' + '.csv')

## ## Visualize results
df_results.head()

,0,1,Inference,id
0,1.295872,-1.186037,0,6978
1,0.835166,-0.908593,0,6979
2,-1.607499,1.858016,1,6980
3,-0.430777,1.007864,1,6981
4,2.083438,-2.189569,0,6982


#Task2 - Multiclass

# Util when the process stops sandly

In [ ]:
# import pickle
# with open('drive/MyDrive/Code/EXITS/Machine-Learning-Tweets-Classification/Bert/Results/EnglishBert_enDataTrain/EnglishBertTask1enDataTrain_Results' + ".pkl", "rb") as f:
#   Re = pickle.load(f)
# Re

In [ ]:
# DfResultsTask = pd.read_csv(Path + 'AverageSpanishBertTask1Results_CSV_.csv', index_col=[0,1], skipinitialspace=True)

In [ ]:
# def CleanBrokeTrain(FileName, Path, NumberOfFoldes=10):
#   with open(Path + FileName + ".pkl", "rb") as f:
#               Results = pickle.load(f)

#   for BT, ModelBertType,  in Results.items():
#     for OP, OutPut in ModelBertType.items():
#       for LR, LearningRate in OutPut.items():
#         for BS, BatchSize in LearningRate.items():
#           for EP, Epoch in BatchSize.items():
#             for Metrics, ValuesCrossValidation in  Epoch.items():
 
#               if len(ValuesCrossValidation) != 0 and not len(ValuesCrossValidation) == NumberOfFoldes:
#                 Results[BT][OP][LR][BS][EP][Metrics] = []
            
#   with open(FileName + '.pkl','wb') as f:
#     pickle.dump(Results, f)

#   with open(Path + FileName + '.pkl','wb') as f:
#     pickle.dump(Results, f)

In [ ]:
# CleanBrokeTrain(FileName=FileResults, Path=Path, NumberOfFoldes=10)

In [ ]:
# LengPhrase = df_train['text'].str.split().str.len().tolist()
# LengPhrase.sort()
# LengPhrase[-13:]

In [ ]:
# LengPhrase = df_RealData['text'].str.split().str.len().tolist()
# LengPhrase.sort()

# LengPhrase[-50:]